In [18]:
# Import libraries 
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, date, timedelta
from pandas.tseries.offsets import MonthBegin, MonthEnd
import pyarrow
import dask
import dask.dataframe as dd
import utilities as nori
from utilities import *

# Connect to ODBC
import pyodbc
conn = pyodbc.connect(DSN='Radar',UID='ruser',PWD='8uzUTHub')

# Date
currentDt = pd.datetime.now().date()
date1 = (currentDt - (pd.Timedelta(pd.offsets.Day(365))) + MonthBegin(-2)).strftime('%m/%d/%Y 00:0:0')
date2 = (currentDt - (pd.Timedelta(pd.offsets.Day(366))) + MonthEnd(6)).strftime('%m/%d/%Y 00:0:0')

transDt1 = (currentDt - pd.Timedelta(pd.offsets.Day(367))).strftime('%m/%d/%Y 00:0:0')
transDt2 = (currentDt - pd.Timedelta(pd.offsets.Day(374))).strftime('%m/%d/%Y 00:0:0')
transDt3 = (currentDt - pd.Timedelta(pd.offsets.Day(381))).strftime('%m/%d/%Y 00:0:0')

# Published Fare
query1 = """SELECT
  REVENUE.RecordLocator AS PNR,
  TRVL_CALENDAR.DateDate AS TrvlDt,
  BKG_CALENDAR.DateDate AS BkDt,
  AIRPORT_POO.AirportCode AS Org,
  AIRPORT_POD.AirportCode AS Dest,
  AIRPORT_POO.IsDomestic AS OrgDom,
  AIRPORT_POD.IsDomestic AS DestDom,
  AIRPORT_LEG_DEP.AirportCode AS Dept,
  AIRPORT_LEG_ARL.AirportCode AS Arr,
  (ROUTE_LEG.IndustryMiles) AS Miles,
  count( distinct REVENUE.PassengerSrcID ) AS Pax,
  Sum(REVENUE.RevenueAmount) AS Rev
FROM
  CalendarDim  BKG_CALENDAR INNER JOIN RevenueFact  REVENUE ON (REVENUE.BookingDateSID=BKG_CALENDAR.DateSID)
   INNER JOIN ClassOfServiceDim  CLASSOFSERVICE ON (REVENUE.ClassOfServiceSID=CLASSOFSERVICE.ClassOfServiceSID  AND  CLASSOFSERVICE.IsCurrent=1)
   INNER JOIN TransactionStatusDim  TRANS_STATUS ON (REVENUE.TransactionStatusSID=TRANS_STATUS.TransactionStatusSID)
   INNER JOIN TransactionTypeDim  TRANS_TYPE ON (TRANS_TYPE.TransactionTypeSID=REVENUE.TransactionTypeSID)
   INNER JOIN CalendarDim  TRVL_CALENDAR ON (REVENUE.DepartureDateSID=TRVL_CALENDAR.DateSID)
   INNER JOIN RouteDim  ROUTE_LEG ON (REVENUE.LegRouteSID=ROUTE_LEG.RouteSID  AND  ROUTE_LEG.IsCurrent=1)
   INNER JOIN AirportDim  AIRPORT_LEG_DEP ON (ROUTE_LEG.RouteDeprAirportSID=AIRPORT_LEG_DEP.AirportSID)
   INNER JOIN AirportDim  AIRPORT_LEG_ARL ON (ROUTE_LEG.RouteArrvAirportSID=AIRPORT_LEG_ARL.AirportSID)
   INNER JOIN AirportDim  AIRPORT_POO ON (AIRPORT_POO.AirportSID=REVENUE.POO_SID)
   INNER JOIN AirportDim  AIRPORT_POD ON (AIRPORT_POD.AirportSID=REVENUE.POD_SID)
   INNER JOIN ChannelDim  BKG_CHANNEL ON (BKG_CHANNEL.ChannelSID=REVENUE.BookingChannelSID)
  
WHERE
  (
   TRVL_CALENDAR.DateDate  BETWEEN  '%s'  AND  '%s'
   AND
   BKG_CALENDAR.DateDate <= '%s'
   AND
   AIRPORT_POO.AirportCode  <>  'UNK'
   AND
   AIRPORT_POD.AirportCode  <>  'UNK'
   AND
   TRANS_STATUS.StatusTier3  IN  ( 'Balance Due','Boarded (flown)','Booked','Expired','No Show','Unknown'  )
   AND
   TRANS_TYPE.TypeTier3  =  'Published Fare'
   AND
   CLASSOFSERVICE.bool_NonRev  =  0
  )
GROUP BY
  REVENUE.RecordLocator, 
  TRVL_CALENDAR.DateDate, 
  BKG_CALENDAR.DateDate, 
  AIRPORT_POO.AirportCode, 
  AIRPORT_POD.AirportCode, 
  AIRPORT_POO.IsDomestic, 
  AIRPORT_POD.IsDomestic, 
  AIRPORT_LEG_DEP.AirportCode, 
  AIRPORT_LEG_ARL.AirportCode, 
  (ROUTE_LEG.IndustryMiles)
"""%(date1,date2,transDt1)
df1 = pd.read_sql(query1,conn,parse_dates=['TrvlDt','BkDt'])

df1 = nori.connect(df1) # Connects
df1 = df1.groupby(['PNR','TrvlDt','BkDt','Org','Dest','OrgDom','DestDom'],as_index=False).agg({'Pax':'max','Rev':'sum','Connect':'max','Miles':'sum'})

# Baggage
query2 = """SELECT
  REVENUE.RecordLocator AS PNR,
  TRVL_CALENDAR.DateDate AS TrvlDt,
  TRANS_CALENDAR.DateDate AS TransDt,
  AIRPORT_POO.AirportCode AS Org,
  AIRPORT_POD.AirportCode AS Dest,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG1' THEN REVENUE.PassengerSrcID ELSE NULL END) as Bag1,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG1' THEN REVENUE.RevenueAmount ELSE 0 END) as Bag1Rev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG2' THEN REVENUE.PassengerSrcID ELSE NULL END) as Bag2,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 = 'BAG2' THEN REVENUE.RevenueAmount ELSE 0 END) as Bag2Rev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 IN ( 'Gate Carry-On','Carry-On' ) THEN REVENUE.PassengerSrcID ELSE NULL END) as CarryOn,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 IN ( 'Gate Carry-On','Carry-On' ) THEN REVENUE.RevenueAmount ELSE 0 END) as CarryOnRev,
  COUNT(DISTINCT CASE WHEN TRANS_TYPE.TypeTier3 IN ('Gate Carry-On','Carry-On','BAG1','BAG2','BAG3','BAG4','BAG5') THEN REVENUE.PassengerSrcID ELSE NULL END) as Cum,
  SUM(CASE WHEN TRANS_TYPE.TypeTier3 IN ('Gate Carry-On','Carry-On','BAG1','BAG2','BAG3','BAG4','BAG5') THEN REVENUE.RevenueAmount ELSE 0 END) as CumRev

FROM
  ChannelDim  TRANS_CHANNEL INNER JOIN RevenueFact  REVENUE ON (REVENUE.TransactionChannelSID=TRANS_CHANNEL.ChannelSID)
   INNER JOIN ClassOfServiceDim  CLASSOFSERVICE ON (REVENUE.ClassOfServiceSID=CLASSOFSERVICE.ClassOfServiceSID  AND  CLASSOFSERVICE.IsCurrent=1)
   INNER JOIN TransactionStatusDim  TRANS_STATUS ON (REVENUE.TransactionStatusSID=TRANS_STATUS.TransactionStatusSID)
   INNER JOIN TransactionTypeDim  TRANS_TYPE ON (TRANS_TYPE.TransactionTypeSID=REVENUE.TransactionTypeSID)
   INNER JOIN CalendarDim  TRVL_CALENDAR ON (REVENUE.DepartureDateSID=TRVL_CALENDAR.DateSID)
   INNER JOIN CalendarDim  TRANS_CALENDAR ON (REVENUE.TransactionDateSID=TRANS_CALENDAR.DateSID)
   INNER JOIN RouteDim  ROUTE_LEG ON (REVENUE.LegRouteSID=ROUTE_LEG.RouteSID  AND  ROUTE_LEG.IsCurrent=1)
   INNER JOIN AirportDim  AIRPORT_LEG_DEP ON (ROUTE_LEG.RouteDeprAirportSID=AIRPORT_LEG_DEP.AirportSID)
   INNER JOIN AirportDim  AIRPORT_LEG_ARL ON (ROUTE_LEG.RouteArrvAirportSID=AIRPORT_LEG_ARL.AirportSID)
   INNER JOIN AirportDim  AIRPORT_POO ON (AIRPORT_POO.AirportSID=REVENUE.POO_SID)
   INNER JOIN AirportDim  AIRPORT_POD ON (AIRPORT_POD.AirportSID=REVENUE.POD_SID)
  
WHERE
  (
   TRVL_CALENDAR.DateDate  BETWEEN  '%s'  AND  '%s'
   AND
   TRANS_CALENDAR.DateDate <= '%s'
   AND
   AIRPORT_POO.AirportCode  <>  'UNK'
   AND
   AIRPORT_POD.AirportCode  <>  'UNK'
   AND
   TRANS_STATUS.StatusTier3  IN  ( 'Balance Due','Boarded (flown)','Booked','Expired','No Show','Unknown'  )
   AND
   TRANS_TYPE.TypeTier3  IN  ( 'BAG1','BAG2','BAG3','BAG4','BAG5','Carry-On','Gate Carry-On'  )
   AND
   rtrim(REVENUE.FeeCode)  NOT IN  ( 'BMLC','BML1','BML2','BAGE','BAGN','BAGP'  )
   AND
   CLASSOFSERVICE.bool_NonRev  =  0
  )
GROUP BY
  REVENUE.RecordLocator, 
  TRVL_CALENDAR.DateDate,
  TRANS_CALENDAR.DateDate,
  AIRPORT_POO.AirportCode, 
  AIRPORT_POD.AirportCode 
HAVING
  Sum(REVENUE.RevenueAmount)  >=  0
"""%(date1,date2,transDt1)
df2 = pd.read_sql(query2,conn,parse_dates=['TrvlDt','TransDt'])

ddf1 = df1[(df1.BkDt <= transDt1)] # Conditional selection by booking and transaction date
ddf2 = df1[(df1.BkDt <= transDt2)]
ddf3 = df1[(df1.BkDt <= transDt3)]

ddf4 = df2[(df2.TransDt <= transDt1)] 
ddf5 = df2[(df2.TransDt <= transDt2)]
ddf6 = df2[(df2.TransDt <= transDt3)]

df1 = pd.merge(ddf1,ddf4,how='left',on=['PNR','TrvlDt','Org','Dest']) # Merge and convert to Dask
df2 = pd.merge(ddf2,ddf5,how='left',on=['PNR','TrvlDt','Org','Dest'])
df3 = pd.merge(ddf3,ddf6,how='left',on=['PNR','TrvlDt','Org','Dest'])

df1,df2,df3 = [dd.from_pandas(df,npartitions=3) for df in [df1,df2,df3]]

# Data Wrangling
df1,df2,df3 = [df.pipe(month) for df in [df1,df2,df3]] # Month Extract
df1,df2,df3 = [df.pipe(diff) for df in [df1,df2,df3]] # DTD
df1,df2,df3 = [df.pipe(journey) for df in [df1,df2,df3]] # Domestic Indicator 

df1,df2,df3 = [df.assign(Domestic = df.Domestic.apply(domestic,meta='bool')) for df in [df1,df2,df3]]

trip1,trip2,trip3 = [df.pipe(duration) for df in [df1,df2,df3]] # Trip Duration
df1 = df1.merge(trip1,how='left',on='PNR')
df2 = df2.merge(trip2,how='left',on='PNR')
df3 = df3.merge(trip3,how='left',on='PNR')

df1,df2,df3 = [df.assign(Miles_Group = df.Miles.apply(miles,meta='object')) for df in [df1,df2,df3]] # Feature binning
df1,df2,df3 = [df.assign(DTD_Group = df.DTD.apply(dtd,meta='object')) for df in [df1,df2,df3]]
df1,df2,df3 = [df.assign(Pax_Group = df.Pax.apply(pax,meta='object')) for df in [df1,df2,df3]]

df1,df2,df3 = [df.pipe(cluster) for df in [df1,df2,df3]] # Cluster
df1,df2,df3 = [df.fillna(0) for df in [df1,df2,df3]] # Fill missing values

df1Total, df2Total, df3Total = [
    df.groupby(
        ['TrvlMo']
    ).agg({'Connect':'count'}).reset_index().rename(
        columns={'Connect':'ConnectTotal'}) for df in [df1,df2,df3]]

df1 = nori.group([df1,df1Total])
df2 = nori.group([df2,df2Total])
df3 = nori.group([df3,df3Total])

# Groupby
df1, df2, df3 = [
    df.groupby(
        ['Cluster','Miles_Group','DTD_Group','Pax_Group','TrvlMo','Domestic']
    ).agg({'PNR':'count','Pax':'sum','Rev':'mean','Duration':'mean',
           'Connect':'sum', 'ConnectTotal':'count','CarryOn':'sum','CarryOnRev':'sum','Bag1':'sum',
           'Bag1Rev':'sum','Bag2':'sum','Bag2Rev':'sum','Cum':'sum','CumRev':'sum'}).reset_index() for df in [df1,df2,df3]]

df1Total, df2Total, df3Total = [
    df.groupby(
        ['TrvlMo']
    ).agg({'PNR':'sum'}).reset_index().rename(
        columns={'PNR':'PNRTotal'}) for df in [df1,df2,df3]]

df1 = nori.group([df1,df1Total])
df2 = nori.group([df2,df2Total])
df3 = nori.group([df3,df3Total])

# Key metrics
df1,df2,df3= [df.pipe(ratios) for df in [df1,df2,df3]] 
df1,df2,df3= [df.pipe(bag1) for df in [df1,df2,df3]]
df1,df2,df3= [df.pipe(bag2) for df in [df1,df2,df3]]
df1,df2,df3= [df.pipe(carryon) for df in [df1,df2,df3]]
df1,df2,df3= [df.pipe(cum) for df in [df1,df2,df3]]

df1.to_parquet('ly {}.parquet'.format(currentDt.strftime('%Y-%m-%d')))  # Save to parquet
df2.to_parquet('lyw {}.parquet'.format(currentDt.strftime('%Y-%m-%d')))
df3.to_parquet('lytw {}.parquet'.format(currentDt.strftime('%Y-%m-%d')))